In [2]:
# Importing of Packages

from bs4 import BeautifulSoup
import re
import itertools
import pandas as pd
import re
import chardet
import numpy as np
import nltk
from nltk.corpus import stopwords
import requests
site = "https://fbookshelf.herokuapp.com"
page = requests.get(site)
print("Importing of Files Done")

C:\Users\cheekushivam\Anconda_file\lib\site-packages\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.23) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


Importing of Files Done


In [3]:
# Scraping Data from this Site "https://fbookshelf.herokuapp.com/#" and converting into pandas DataFrame

columnNames = ['Title','Author','Genre','NumberofPages','StarRating','NumberofReviews','ShortDescription','ISBNNumber','NumbeofVotes','Comments']
hackathonDF = pd.DataFrame(columns = columnNames)

titleArray=[]
authorArray=[]
genreArray=[]
pagesArray=[]
starRatingArray=[]
descriptionArray=[]
reviewsArray=[]
isbnArray=[]
votesArray=[]
commentsArray=[]
commentsInternalData=""
soup = BeautifulSoup(page.content, 'html.parser')
contentDiv = soup.find_all(class_="col-md-8")
for content in contentDiv:
    title = content.find('h3').get_text()
    titleArray.append(title)
    authorby = content.find(class_="author").get_text()
    author = authorby.split("by")[1]
    authorArray.append(author)
    genre = content.find(class_="genre").get_text()
    genreorg = genre.split("genre :")[1]
    genreArray.append(genreorg)
    pages = content.find(class_="pages").get_text()
    pagesorg = pages.split("pages")[0]
    pagesArray.append(pagesorg)
    starRating = content.find(class_="rating").get_text()
    starRatingArray.append(starRating)
    numOfReviews = content.find(class_="#reviews").get_text()
    numOfReviewsorg = numOfReviews.split("No_of_reviews")[1]
    reviewsArray.append(numOfReviewsorg)
    description = content.find(class_="description").get_text()
    descriptionArray.append(description)
    isbn = content.find(class_="isbn").get_text()
    isbnorg = isbn.split("ISBN")[1]
    isbnArray.append(isbnorg)
    votes = content.find(class_="votes").get_text()
    votesorg = votes.split("votes")[0]
    votesArray.append(votesorg)
    comments = content.find_all('p', attrs={'class': re.compile("^c[0-9]*")})
    if len(comments) == 0 :
        commentsArray.append("None")
        continue
    else:
        for comment in comments:
                commentsInternalData += comment.get_text()
                commentsInternalData += "|"        
        commentsArray.append(commentsInternalData)
        commentsInternalData = ""
    
hackathonDF['Title'] = titleArray
hackathonDF['Author'] = authorArray
hackathonDF['Genre'] = genreArray
hackathonDF['NumberofPages'] = pagesArray
hackathonDF['StarRating'] = starRatingArray
hackathonDF['NumberofReviews'] = reviewsArray
hackathonDF['ShortDescription'] = descriptionArray
hackathonDF['ISBNNumber'] = isbnArray
hackathonDF['NumbeofVotes'] = votesArray
hackathonDF['Comments'] = commentsArray
export_csv = hackathonDF.to_csv (r'scraping_hackathon_dataFinal.csv', index = None, header=True)
print("Data sent To CSV")

Data sent To CSV


In [4]:
# Finding out What's the Encoding of Data Whichwe got from Site.

with open("scraping_hackathon_dataFinal.csv", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(10000))
#print(result)

In [5]:
# Getting from CSV for Data Exploration using Encoding "Windows-1252"

scraping_data = pd.read_csv('scraping_hackathon_dataFinal.csv',encoding="Windows-1252")
print("Data Imported")

Data Imported


In [6]:
# Performing Data Exploration
resultClassics = (scraping_data.Genre.values == ' Classics').sum()
#print(resultClassics) #252
resultSF = (scraping_data.Genre.values == ' Science Fiction').sum()
#print(resultSF) #59

# Calculating Top 10 Genres
resultUniqueGenres = scraping_data.Genre.unique()
#print(len(resultUniqueGenres)) #101
genreCount = []
columnNamesGenre  = ['Genre','Count']
genreWithCount = pd.DataFrame(columns = columnNamesGenre)
genCount = 0
for genre in resultUniqueGenres:
    genCount = (scraping_data.Genre.values == genre).sum()
    genreCount.append(genCount)
genreWithCount.Genre = resultUniqueGenres
genreWithCount.Count = genreCount
genreWithCount = genreWithCount.sort_values(by=['Count'],ascending=False)
genreWithCount = genreWithCount.head(10)
#print(genreWithCount)
print("Count Done")

Count Done


In [7]:
# Authors appearing maximum number of genres
columnNamesGenreWithAuthor  = ['Author','Genre']
genreSet = ""
authorSet = ""
genreSetArray = []
authorSetArray = []
authorWithGenreCount = pd.DataFrame(columns = columnNamesGenreWithAuthor)
for index,row in scraping_data.iterrows():
    for genreUniqueOne in genreWithCount.Genre:
        if row.Genre == genreUniqueOne:
            genreSet = genreUniqueOne
            authorSet = row.Author
            genreSetArray.append(genreSet)
            authorSetArray.append(authorSet)
authorWithGenreCount.Genre = genreSetArray
authorWithGenreCount.Author = authorSetArray
#print(authorWithGenreCount)
authorWithGenreCount = authorWithGenreCount.sort_values(by=['Genre'])
authorWithGenreCount = authorWithGenreCount.drop_duplicates(subset=['Author','Genre'],keep = "first")
export_csv = authorWithGenreCount.to_csv (r'scraping_hackathon_dataWithAuthorandGenre.csv', index = None, header=True)
#print(authorWithGenreCount)
print("Count Done")

Count Done


In [8]:
# Range of Ratings Under Category Classics Genre
resultClassics = scraping_data.loc[scraping_data.Genre == ' Classics']
#print(resultClassics.StarRating.max())
#print(resultClassics.StarRating.min())
# ratings would be between 2.91 and 4.79
print("Done")

Done


In [9]:
# Median of Ratings Under Category Classics Fantasy
resultFantasy = scraping_data.loc[scraping_data.Genre == ' Fantasy']
#print(resultFantasy.StarRating.median())
# Median : 4.295

# Number of votes required for the high rated book in fantasy
resultFantasy = resultFantasy.loc[resultFantasy.StarRating.idxmax]
#print(resultFantasy.NumbeofVotes)
# votes : 38476

#print(resultFantasy.StarRating.max()) #4.78
print("Done")

Done


In [10]:
# Standard Deviation of the Rating of books under Business
resultBusiness = scraping_data.loc[scraping_data.Genre == ' Business']
resultStdDeviation = resultBusiness.StarRating.std()
#print(resultStdDeviation)
# std : 0.35252665700363184
print("Done")

Done


In [12]:
# What is the genre of the highest voted book
columnNamesGenreWithAuthor  = ['Genre','MaxVotes']
maxVotesWithGenre = pd.DataFrame(columns = columnNamesGenreWithAuthor)
maxVotedSet = scraping_data
maxVotedSet.NumbeofVotes = maxVotedSet.NumbeofVotes.replace({',': ''}, regex=True)
maxVotedSet.NumbeofVotes = pd.to_numeric(maxVotedSet.NumbeofVotes)
maxDataset = maxVotedSet.sort_values(['NumbeofVotes'],ascending=False).groupby('Genre').head(10)
maxVotesWithGenre.Genre = maxDataset.Genre
maxVotesWithGenre.MaxVotes = maxDataset.NumbeofVotes
print(maxDataset.Genre.head(1))
#Answer : Fantasy

2097     Fantasy
Name: Genre, dtype: object


In [13]:
# Which genre has more no of authors with above average ratings
avg_rating = scraping_data.StarRating.mean()
dataWithAboveAvgRating = scraping_data.loc[scraping_data.StarRating > avg_rating].groupby('Genre',as_index=False).count().sort_values(['StarRating'],ascending=False).head(1)
print(dataWithAboveAvgRating.Genre)
#Answer None

49     None
Name: Genre, dtype: object


In [34]:
# Which Genre has the author with the highest Std. Deviation in number of pages
pagesSet = scraping_data
pagesSet.NumberofPages = pagesSet.NumberofPages.replace({None : 0.0}, regex=True)
pagesSet.NumberofPages = pd.to_numeric(pagesSet.NumberofPages)
stdDeviationSet = pagesSet.groupby('Genre').NumberofPages.std(ddof=0).sort_values(ascending=False).head(1)
print(stdDeviationSet.index[0])
#Answer Fantasy

<class 'pandas.core.series.Series'>
 Fantasy


In [ ]:
# Among All the available description of books which word in the descriptions has the frequency greater than 6000 and less than 7000
wordsArrayCombine = []
wordsFramePre = pd.DataFrame(columns=['Word'])
wordsFramePost = pd.DataFrame(columns=['Word','Frequency'])
for word in scraping_data.ShortDescription:
        wordsArrayCombine.append(re.findall('\w+',word))
wordsArrayIndividual  = list(itertools.chain(*wordsArrayCombine))
wordsFramePre.Word = wordsArrayIndividual
uniqueWords = wordsFramePre.Word.unique()
wordsFramePost.Word = uniqueWords
count = 0
wordUnique = ""
wordSame = ""

wordArray=[]
frequencyArray=[]
#print(wordsFramePost.Word)

for wordUnique in wordsFramePost.Word:
    for wordSame in wordsFramePre.Word:
        if wordUnique == wordSame:
            count = count + 1
    frequencyArray.append(count)
    wordArray.append(wordUnique)
    count = 0
wordsFramePost.Frequency = frequencyArray
wordsFramePost.Word = wordArray
#print(wordsFramePost.sort_values(ascending = False,by=['Frequency']).head(1))  
#Answer is "The"